# Quantum Gaussian Mixture Clustering

This file contains a Python implementation of the paper "Quantum Clustering and Gaussian Mixtures" by Mahajabin Rahman and Davi Geiger. This project utilizes quantum wave functions and complex numbers to enhance Gaussian mixture clustering, outperforming traditional methods in terms of accuracy and robustness.  

In this notebook i provided a template for you to test three clustering model on your various data.   

In [1]:
# import requred libraries
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import warnings
import random
import cv2
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans

warnings.filterwarnings('ignore')

In [ ]:
# define a function to add noise to image
# Function references = https://stackoverflow.com/questions/22937589/how-to-add-noise-gaussian-salt-and-pepper-etc-to-image-in-python-with-opencv
def sp_noise(image,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output

In [ ]:
# define QuantumGaussianMixture Model 
class QuantumGaussianMixture:
    def __init__(self, n_components, compare=False, n_iter=100, tol=1e-6):
        self.n_components = n_components
        self.n_iter = n_iter
        self.tol = tol
        self.compare = compare 

    def initialize_parameters(self, X):
        n_samples, n_features = X.shape

        # K-means++ initialization for means
        self.mu = np.empty((self.n_components, n_features))
        self.mu[0] = X[np.random.choice(n_samples)]
        
        for k in range(1, self.n_components):
            distances = np.min([np.sum((X - self.mu[j]) ** 2, axis=1) for j in range(k)], axis=0)
            probs = distances / np.sum(distances)
            self.mu[k] = X[np.random.choice(n_samples, p=probs)]
        
        # Initialize alpha uniformly
        self.alpha = np.ones(self.n_components) / self.n_components
        
        # Initialize covariance matrices to the covariance of the dataset
        self.C = np.array([np.cov(X.T) for _ in range(self.n_components)])
        
        # Initialize phases uniformly between 0 and 2*pi
        self.phi = np.random.uniform(0, 2 * np.pi, self.n_components)

    def wave_function(self, x, mu, C, phi):
        d = len(x)
        if d == 1: # If dimension of dataset was 1 it means we have 1 feature and then we do not have covariance matrix instead we have scaler variance 
            C_inv = C ** -1
            Z_k = (2 * np.pi)**(d / 2) * C**0.5
        elif d > 1:
            C_inv = np.linalg.inv(C)
            Z_k = (2 * np.pi)**(d / 2) * np.linalg.det(C)**0.5
            
        diff = x - mu
        exp_part = np.exp(-0.25 * np.dot(diff.T, np.dot(C_inv, diff)))
        phase_part = np.exp(-1j * phi)
        return exp_part * phase_part / Z_k

    def probability(self, x, mu, C):
        d = len(x)
        if d == 1:
            C_inv = C ** -1
            Z_k = (2 * np.pi)**(d / 2) * C**0.5
        elif d > 1:
            C_inv = np.linalg.inv(C)
            Z_k = (2 * np.pi)**(d / 2) * np.linalg.det(C)**0.5
            
        diff = x - mu
        exp_part = np.exp(-0.5 * np.dot(diff.T, np.dot(C_inv, diff)))
        return exp_part / Z_k

    def mixture_wave_function(self, x):
        psi = sum(self.alpha[k] * self.wave_function(x, self.mu[k], self.C[k], self.phi[k])
                  for k in range(self.n_components))
        return psi

    def mixture_probability(self, x):
        psi = self.mixture_wave_function(x)
        return np.abs(psi)**2

    def e_step(self, X):
        n_samples = X.shape[0]
        Q = np.zeros((n_samples, self.n_components))

        for i in range(n_samples):
            total_prob = self.mixture_probability(X[i])
            for k in range(self.n_components):
                Q[i, k] = self.alpha[k] * self.probability(X[i], self.mu[k], self.C[k])
            Q[i, :] /= total_prob
        return Q

    def m_step(self, X, Q):
        n_samples, n_features = X.shape

        for k in range(self.n_components):
            N_k = np.sum(Q[:, k], axis=0)
            self.alpha[k] = N_k / n_samples
            self.mu[k] = np.sum(X * Q[:, k, np.newaxis], axis=0) / N_k
            diff = X - self.mu[k]
            self.C[k] = np.dot((Q[:, k, np.newaxis] * diff).T, diff) / N_k
            self.phi[k] = np.angle(np.sum(Q[:, k] * np.exp(1j * self.phi[k])))

    def fit(self, X):
        if not self.compare:
            self.initialize_parameters(X)

        for iteration in range(self.n_iter):
            Q = self.e_step(X)
            prev_mu = self.mu.copy()
            self.m_step(X, Q)

            if np.linalg.norm(self.mu - prev_mu) < self.tol:
                break

    def predict(self, X):
        Q = self.e_step(X)
        return np.argmax(Q, axis=1)

    def predict_proba(self, X):
        return self.e_step(X)

In [ ]:
# Step 1: Read the image
image_path = '<Your image path>'  # Replace with the path to your image
image = cv2.imread(image_path)
resized_image = cv2.resize(image, (300, 300)) 
noise_img = sp_noise(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB),0.05)
# Optional: Display the original image
plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.show()

plt.imshow(noise_img)
plt.title("Noisy Image")
plt.show()

# Step 2: Preprocess the image

gray_image = cv2.cvtColor(noise_img, cv2.COLOR_BGR2GRAY)

# Reshape the image to a 2D array of pixels
# Each row is a pixel value
pixel_values = gray_image.reshape((-1, 1))

# Convert to float 
pixel_values = np.float64(pixel_values)
n_components = 2  # Number of components (clusters), you can change this value

# implement this function to initialize parameters using kmeans++ for comparability to other algorithms 
def initialize_parameters_kmeans(X, n_components):
    kmeans = KMeans(n_clusters=n_components, init='k-means++', random_state=42).fit(X)
    mu = kmeans.cluster_centers_
    alpha = np.ones(n_components) / n_components
    C = np.array([np.cov(X.T) for _ in range(n_components)])
    phi = np.random.uniform(0, 2 * np.pi, n_components)
    return mu, alpha, C, phi

mu, alpha, C, phi = initialize_parameters_kmeans(pixel_values, n_components)

# Initialize Quantum Gaussian Mixture model
# we set compare to true due to goal of comparison between algorithms
qgm = QuantumGaussianMixture(n_components=n_components, compare=True, n_iter=1000, tol=1e-7)
# Set parameters
qgm.mu = mu.copy()
qgm.alpha = alpha.copy()
qgm.C = C.copy()
qgm.phi = phi.copy()

# Initialize Gaussian Mixture model
gm_model = GaussianMixture(n_components=n_components, max_iter=1000, tol=1e-7, covariance_type='full', random_state=None)
# Set parameters
gm_model.means_init = mu.copy()
gm_model.weights_init = alpha.copy()
gm_model.covariances_init = C.copy()

# Initialize KMeans
kmeans = KMeans(n_clusters=n_components, tol=1e-7, max_iter=1000, random_state=None)
# Set parameters
kmeans.cluster_centers_ = mu.copy()

# Step 3: Fit the models
# Fit the Gaussian Mixture model
gm_model.fit(pixel_values)

# Fit the Quantum Gaussian Mixture model
qgm.fit(pixel_values)

# Fit the KMeans
kmeans.fit(pixel_values)

# Get the labels for each pixel
# Labels of the Gaussian Mixture model
labels = gm_model.predict(pixel_values)
labels = labels.reshape(gray_image.shape)

# Labels of the Quantum Gaussian Mixture model
labelsqgm = qgm.predict(pixel_values)
labelsqgm = labelsqgm.reshape(gray_image.shape)

# Labels of the Kmeans model
labelsk = kmeans.predict(pixel_values)
labelsk = labelsk.reshape(gray_image.shape)

# Display the clustered image
# GMM
plt.imshow(labels, cmap='gray')
plt.title("Segmented Image GMM")
plt.show()

# QGM
plt.imshow(labelsqgm, cmap='gray')
plt.title("Segmented Image QGMM")
plt.show()

# KMeans
plt.imshow(labelsk, cmap='gray')
plt.title("Segmented Image Kmeans")
plt.show()

# Retrieve learned parameters during EM Algorithm

# GMM
print("Means GMM:", gm_model.means_)
print("Covariances GMM:", gm_model.covariances_)
print('---'*10)

# QGMM
print("Means QGMM:", qgm.mu)
print("Covariances QGMM:", qgm.C)
print('---'*10)

# KMeans
print("Means KMeans:", kmeans.cluster_centers_)
